In [88]:
#data science stuff
import numpy as np
import pandas as pd
import configparser
config = configparser.ConfigParser()
config.read('config.INI')

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

%run readTeams.ipynb
%run osuApiRequests.ipynb

#requests
import requests

Sources of help:
https://davidhamann.de/2017/06/26/pandas-select-elements-by-string/

In [166]:
# Function: processMP
# Processes the match corresponding to the mp_id passed in by: updating the
# stage leaderboards, the map counts dict, and the team info
# params: map_id - the id of the match we wish to process
#         leaderboards_in_stage - leaderboards that will be updated
#         map_counts_in_stage - dict of the times each map in the stage was played
#         map_mods - two col df; 1st col is map_id, 2nd col is mod type
# return: void
def processMP(mp, leaderboards_in_stage, map_counts_in_stage, is_team_tourney, 
              teams, desired_scoring_type, mods_for_each_map):

    get_match_response = get_match(mp)
    
    games = get_match_response['games']
    #iterate through the maps played in games
    for i in np.arange(len(games)):
        #print("i: " + str(i))
        #---Error checking---
        #wrong scoring type
        if games[i]['scoring_type'] != desired_scoring_type:
            print('     1st err: ' + desired_scoring_type + " " + mp)
            continue
        #no scores were recorded
        elif len(games[i]['scores']) == 0:
            print('     2nd err')
            continue
        #map played isn't in the pool of the current stage
        elif games[i]['beatmap_id'] not in map_counts_in_stage:
            print('     3rd err')
            continue
        #map's mods are not in the list of playable mods
        elif games[i]['mods'] not in list(mods_for_each_map.values()):
            print('     4th err')
            continue
        
        #---Update map_counts_in_stage---
        map_counts_in_stage[games[i]['beatmap_id']] += 1
        
        #---Insert new entries into leaderboard---
        for j in np.arange(len(games[i]['scores'])):
            #print("   j: " + str(j))
            current_player = games[i]['scores'][j]
            
            #error check for referee
            if current_player['score'] == '0':
                continue
            
            #get player name
            get_user_response = \
                get_user(current_player['user_id'], games[i]['play_mode'], 'id')
            if len(get_user_response) == 0:
                continue
            username = get_user_response[0]['username']
            #print(username)
            
            
            #determine mod used by using the mp_ids sheet because the api
            #isn't very informative on this
            mod = mods_for_each_map[games[i]['beatmap_id']]

            if is_team_tourney == True:
                new_row = [{
                    '#': len(leaderboards_in_stage[games[i]['beatmap_id']]['scores'].index) + 1, 
                    'Player': username,
                    'Team': getTeam(teams, username),
                    'Rank': calculateRank(current_player, games[i]['beatmap_id']),
                    'Score': current_player['score'],
                    'Combo': current_player['maxcombo'],
                    'Accuracy': calculateAcc(current_player),
                    'Mods': mod,
                    'Match': 'https://osu.ppy.sh/community/matches/' + mp_id
                }]
            else:
                new_row = [{
                    '#': len(leaderboards_in_stage[games[i]['beatmap_id']]['scores'].index) + 1, 
                    'Player': username,
                    'Rank': calculateRank(current_player, games[i]['beatmap_id']),
                    'Score': current_player['score'],
                    'Combo': current_player['maxcombo'],
                    'Accuracy': calculateAcc(current_player),
                    'Mods': mod,
                    'Match': 'https://osu.ppy.sh/community/matches/' + mp_id
                }]
            #print(new_row)
            #print(games[i]['beatmap_id'])
            leaderboards_in_stage[games[i]['beatmap_id']]['scores'] = leaderboards_in_stage[games[i]['beatmap_id']]['scores'].append(new_row, ignore_index = True)
            #print('fuck??')
            #print(leaderboards_in_stage[ games[i]['beatmap_id'] ]['scores'])

    

In [149]:
#calculate acc of player to two decimal places
def calculateAcc(player):
    #convert player's counts to int for calculations
    count50 = int(player['count50'])
    count100 = int(player['count100'])
    count300 = int(player['count300'])
    countmiss = int(player['countmiss'])

    #calculate accuracy
    player_total_objects = count50 + \
                    count100 + \
                    count300 + \
                    countmiss
    acc = 100 * (50*count50 + 100*count100 + 300*count300) \
            / (300*player_total_objects)
    acc = round(acc, 2)
    return acc

In [137]:
#TODO
def calculateRank(player, map_id):
    
    #---Determine if player finished map---
    #get map's total object count used to compare to player's
    get_scores_response = get_scores(map_id, '0', '1')
    map_total_objects = int(get_scores_response[0]['count300']) + \
                        int(get_scores_response[0]['count100']) + \
                        int(get_scores_response[0]['count50']) + \
                        int(get_scores_response[0]['countmiss'])

    #convert player's counts to int for calculations
    count50 = int(player['count50'])
    count100 = int(player['count100'])
    count300 = int(player['count300'])
    countmiss = int(player['countmiss'])
    #player's total object count used to compare to map's and calculate rank
    player_total_objects = count50 + \
                    count100 + \
                    count300 + \
                    countmiss
    
    #set unfinished flag
    unfinished_flag = True if player_total_objects != map_total_objects else False
    
    #---Determine grade---
    #F case 
    if player['pass'] == '0':
        return 'F' if (unfinished_flag == False) else 'F (U)'
    
    prop_300s = count300 / player_total_objects
    #SS case
    if prop_300s == 1:
        return 'SS' if unfinished_flag == False else 'SS (U)'
    
    prop_50s = count50 / player_total_objects
    no_misses = True if countmiss == 0 else False
    #S case
    if (prop_300s > 0.90) & (prop_50s < 0.01) & no_misses:
        return 'S' if unfinished_flag == False else 'S (U)'

    #A-D cases
    if ((prop_300s > 0.80) & no_misses) | (prop_300s > 0.90):
        return 'A' if unfinished_flag == False else 'A (U)'
    if ((prop_300s > 0.70) & no_misses) | (prop_300s > 0.80):
        return 'B' if unfinished_flag == False else 'B (U)'
    if prop_300s > 0.60:
        return 'C' if unfinished_flag == False else 'C (U)'
    return 'D' if unfinished_flag == False else 'D (U)'